In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.tsa.holtwinters import Holt, ExponentialSmoothing, SimpleExpSmoothing
from sklearn import linear_model
pd.options.mode.chained_assignment = None  # default='warn'
from itertools import chain
from scipy.stats import ttest_ind

In [3]:
#Load Base Dataser
demo = pd.read_csv('demographics.csv')
pop = pd.read_csv('pop.csv')
demo.head()

,Year,Country of origin,Country of origin (ISO),Female 0 - 4,Female 5 - 11,Female 12 - 17,Female 18 - 59,Female 60,Female total,Male 0 - 4,Male 5 - 11,Male 12 - 17,Male 18 - 59,Male 60,Male total,Total
0,2001,Afghanistan,AFG,192136,0,0,581881,40530,1385962,195434,0,0,782771,57368,1643930,5109754
1,2001,Albania,ALB,0,0,0,15,0,451,0,0,0,9,0,662,7625
2,2001,Algeria,DZA,0,0,0,34,0,350,11,0,0,93,0,680,8399
3,2001,Angola,AGO,12216,0,0,27384,2117,74574,11412,0,0,22973,1826,73374,672618
4,2001,Antigua and Barbuda,ATG,0,0,0,0,0,0,0,0,0,0,0,0,5


In [4]:
demo_scaled = demo.join(pop.set_index('Country Code'), on = 'Country of origin (ISO)')

In [5]:
df = demo_scaled[['Year', 'Country of origin','Country of origin (ISO)', 'Female total','Male total', 'Total', '2019']]
df.head()

,Year,Country of origin,Country of origin (ISO),Female total,Male total,Total,2019
0,2001,Afghanistan,AFG,1385962,1643930,5109754,38041754.0
1,2001,Albania,ALB,451,662,7625,2854191.0
2,2001,Algeria,DZA,350,680,8399,43053054.0
3,2001,Angola,AGO,74574,73374,672618,31825295.0
4,2001,Antigua and Barbuda,ATG,0,0,5,97118.0


In [6]:
#number of refugees per million people
demo_scaled['female'] = demo_scaled['Female total'] / (demo_scaled['2019']/1000000)
demo_scaled['male'] = demo_scaled['Male total'] / (demo_scaled['2019']/1000000)
demo_scaled['total'] = demo_scaled['Total'] / (demo_scaled['2019']/1000000)


demo_scaled = demo_scaled.rename(columns={'2019':'pop'})
demo_scaled.head()

,Year,Country of origin,Country of origin (ISO),Female 0 - 4,Female 5 - 11,Female 12 - 17,Female 18 - 59,Female 60,Female total,Male 0 - 4,...,Male 12 - 17,Male 18 - 59,Male 60,Male total,Total,Country Name,pop,female,male,total
0,2001,Afghanistan,AFG,192136,0,0,581881,40530,1385962,195434,...,0,782771,57368,1643930,5109754,Afghanistan,38041754.0,36432.652396,43213.832885,134319.621540
1,2001,Albania,ALB,0,0,0,15,0,451,0,...,0,9,0,662,7625,Albania,2854191.0,158.013251,231.939628,2671.510071
2,2001,Algeria,DZA,0,0,0,34,0,350,11,...,0,93,0,680,8399,Algeria,43053054.0,8.129505,15.794466,195.084883
3,2001,Angola,AGO,12216,0,0,27384,2117,74574,11412,...,0,22973,1826,73374,672618,Angola,31825295.0,2343.230440,2305.524584,21134.698044
4,2001,Antigua and Barbuda,ATG,0,0,0,0,0,0,0,...,0,0,0,0,5,Antigua and Barbuda,97118.0,0.000000,0.000000,51.483762


In [7]:
df2= demo_scaled[['Year', 'Country of origin (ISO)', 'total','female','male']]

In [8]:
#GDP PER CAPITA(PPP)
PPP = pd.read_csv('PPP.csv')
PPP= PPP.T.fillna(PPP.mean(axis=1)).T
PPP.head()

,Country Name,ISO,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,20669,20436.9,20833.8,22570,23300,24045.3,25835.1,27084.7,...,23512.6,24986,24713.7,26189.4,26647.9,27980.9,28281.4,29007.7,24748.5,24748.5
1,Afghanistan,AFG,438.082,179.427,190.684,211.382,242.031,263.734,359.693,364.661,...,543.303,591.163,641.871,637.166,613.857,578.466,509.219,519.885,493.75,507.103
2,Angola,AGO,527.334,872.494,982.961,1255.56,1902.42,2599.57,3122,4080.94,...,3587.88,4615.47,5100.1,5254.88,5408.41,4166.98,3506.07,4095.81,3289.65,2790.73
3,Albania,ALB,1281.66,1425.12,1846.12,2373.58,2673.79,2972.74,3595.04,4370.54,...,4094.35,4437.14,4247.63,4413.06,4578.63,3952.8,4124.06,4531.02,5284.38,5353.24
4,Andorra,AND,22971.5,25066.9,32272,37969.2,40066.3,42675.8,47803.7,48718.5,...,40852.7,43335.3,38686.5,39538.8,41303.9,35762.5,37474.7,38962.9,41793.1,40886.4


In [9]:
#GDP
GDP = pd.read_csv('GDP.csv')
GDP = GDP.T.fillna(GDP.mean(axis=1)).T
GDP.head()

,Country Name,ISO,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,1.92011e+09,1.94134e+09,2.02123e+09,2.22849e+09,2.33073e+09,2.42458e+09,2.61508e+09,2.74525e+09,...,2.3905e+09,2.54972e+09,2.53464e+09,2.70168e+09,2.76536e+09,2.91955e+09,2.96592e+09,3.05642e+09,2.50644e+09,2.50644e+09
1,Afghanistan,AFG,1.38012e+10,4.05518e+09,4.51556e+09,5.22678e+09,6.20914e+09,6.97129e+09,9.74788e+09,1.01092e+10,...,1.58566e+10,1.78043e+10,2.00016e+10,2.05611e+10,2.04849e+10,1.99071e+10,1.80177e+10,1.88699e+10,1.83539e+10,1.92911e+10
2,Angola,AGO,8.93606e+09,1.52856e+10,1.78127e+10,2.35521e+10,3.69709e+10,5.2381e+10,6.52665e+10,8.85386e+10,...,8.37995e+10,1.12e+11,1.28e+11,1.37e+11,1.46e+11,1.16e+11,1.01e+11,1.22e+11,1.01e+11,8.88157e+10
3,Albania,ALB,3.9221e+09,4.34807e+09,5.61149e+09,7.18468e+09,8.05208e+09,8.89607e+09,1.06773e+10,1.28814e+10,...,1.19269e+10,1.28908e+10,1.23198e+10,1.27762e+10,1.32281e+10,1.13868e+10,1.18612e+10,1.30197e+10,1.5147e+10,1.52792e+10
4,Andorra,AND,1.54693e+09,1.75591e+09,2.36173e+09,2.89492e+09,3.15991e+09,3.45644e+09,3.9526e+09,4.08563e+09,...,3.44997e+09,3.6292e+09,3.18881e+09,3.1937e+09,3.27181e+09,2.78987e+09,2.89668e+09,3.00018e+09,3.21832e+09,3.15406e+09


In [10]:
#life expentancy
life = pd.read_csv('life expectancy.csv')
life = life.T.fillna(life.mean(axis=1)).T
life.head()

,Country Name,ISO,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,73.787,73.853,73.937,74.038,74.156,74.287,74.429,74.576,...,74.872,75.017,75.158,75.299,75.441,75.583,75.725,75.868,76.01,76.152
1,Afghanistan,AFG,55.841,56.308,56.784,57.271,57.772,58.29,58.826,59.375,...,60.484,61.028,61.553,62.054,62.525,62.966,63.377,63.763,64.13,64.486
2,Angola,AGO,46.522,47.059,47.702,48.44,49.263,50.165,51.143,52.177,...,54.311,55.35,56.33,57.236,58.054,58.776,59.398,59.925,60.379,60.782
3,Albania,ALB,73.955,74.288,74.579,74.828,75.039,75.228,75.423,75.646,...,76.221,76.562,76.914,77.252,77.554,77.813,78.025,78.194,78.333,78.458
4,Andorra,AND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#School enrollment, primary (% gross)
secondary= pd.read_csv('secondary.csv')
secondary = secondary.T.fillna(secondary.mean(axis=1)).T
secondary.head()

,Country Name,ISO,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,109.441,111.011,109.214,110.839,111.377,114.251,115.212,113.092,...,113.794,105.256,104.087,116.039,117.108,111.752,111.752,111.752,111.752,111.752
1,Afghanistan,AFG,20.8829,68.9868,93.3622,104.664,98.0999,101.698,98.9335,100.78,...,100.072,97.6416,103.492,104.498,105.92,103.535,102.486,102.176,103.996,94.8956
2,Angola,AGO,107.846,107.846,107.846,107.846,107.846,107.846,98.9736,105.18,...,105.781,119.53,107.846,107.846,107.846,113.478,107.846,107.846,107.846,107.846
3,Albania,ALB,102.967,101.412,100.005,98.178,100.676,97.8757,96.3456,95.6825,...,93.4905,95.0061,97.5851,99.9839,104.108,105.543,105.554,107.047,106.993,104.813
4,Andorra,AND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#Minize AIC
def minAIC(X,y):
    variables = X.columns
    model = sm.OLS(y,X[variables]).fit()
    while True:
        maxp = np.max(model.pvalues)
        new_variables = variables[model.pvalues < maxp]
        newmodel = sm.OLS(y,X[new_variables]).fit()
        if newmodel.aic < model.aic:
            model = newmodel
            variables = new_variables
        else:
            break
    return model,variables

In [17]:
#Sample
df_ppp = PPP[PPP['ISO'] == 'AFG'].drop(columns=PPP.columns[[0,1]]).T
df_ppp=sum(np.array(df_ppp).tolist(), [])

df_life = life[life['ISO'] == 'AFG'].drop(columns=life.columns[[0,1]]).T
df_life=sum(np.array(df_life).tolist(), [])

df_gdp = GDP[GDP['ISO'] == 'AFG'].drop(columns=GDP.columns[[0,1]]).T
df_gdp=sum(np.array(df_gdp).tolist(), [])

df_secondary = secondary[secondary['ISO'] == 'AFG'].drop(columns=secondary.columns[[0,1]]).T
df_secondary=sum(np.array(df_secondary).tolist(), [])
    
df = df2[df2['Country of origin (ISO)'] == 'AFG']

df['ppp'] = df_ppp
df['gdp'] = df_gdp
df['life'] = df_life
df['secondary'] = df_secondary
df['diff'] = df['male']-df['female']

In [18]:
df

,Year,Country of origin (ISO),total,female,male,ppp,gdp,life,secondary,diff
0,2001,AFG,134319.621540,36432.652396,43213.832885,438.081704,1.380124e+10,55.841,20.882910,6781.180489
172,2002,AFG,86101.576704,17412.577769,15556.669653,179.426611,4.055180e+09,56.308,68.986847,-1855.908116
353,2003,AFG,63622.276723,24225.118537,27725.088596,190.683814,4.515559e+09,56.784,93.362183,3499.970059
536,2004,AFG,70292.947060,24663.163533,26103.528244,211.382117,5.226779e+09,57.271,104.663818,1440.364711
715,2005,AFG,63325.050680,24823.461084,28392.723427,242.031285,6.209138e+09,57.772,98.099892,3569.262343
895,2006,AFG,61437.519416,25186.693547,30161.832181,263.733692,6.971286e+09,58.290,101.697853,4975.138633
1083,2007,AFG,84427.863132,37558.757149,44195.123075,359.693238,9.747880e+09,58.826,98.933510,6636.365926
1268,2008,AFG,80592.524730,36176.013335,41712.456266,364.660745,1.010923e+10,59.375,100.779732,5536.442931
1454,2009,AFG,83742.405779,37892.916294,43823.505089,438.076034,1.243909e+10,59.930,96.896400,5930.588795
1639,2010,AFG,111612.624381,39208.602211,43362.721919,543.303042,1.585657e+10,60.484,100.071709,4154.119708


In [38]:
def myModels(country):
    df_ppp = PPP[PPP['ISO'] == country].drop(columns=PPP.columns[[0,1]]).T
    df_ppp=sum(np.array(df_ppp).tolist(), [])
    
    df_life = life[life['ISO'] == country].drop(columns=life.columns[[0,1]]).T
    df_life=sum(np.array(df_life).tolist(), [])
    
    df_gdp = GDP[GDP['ISO'] == country].drop(columns=GDP.columns[[0,1]]).T
    df_gdp=sum(np.array(df_gdp).tolist(), [])
    
    df_secondary = secondary[secondary['ISO'] == country].drop(columns=secondary.columns[[0,1]]).T
    df_secondary=sum(np.array(df_secondary).tolist(), [])
    
    
    df = df2[df2['Country of origin (ISO)'] == country]
    
    df['ppp'] = df_ppp
    df['gdp'] = df_gdp
    df['life'] = df_life
    df['secondary'] = df_secondary
    df['diff'] = df['male']-df['female']
    
    
    X = df[['Year','ppp','gdp','life','secondary']]
    
    X_train,X_test,y_train,y_test = train_test_split(X,df['total'],random_state=10)
    
    #LASSO
    clf = linear_model.Lasso(alpha=0.05)
    clf.fit(X_train, y_train)
    
    #AIC
    AICmodel,variables = minAIC(X_train, y_train)
    
    
    df['above_mean'] = df['total']> np.mean(df['total'])
    
    logitmodel = sm.Logit(df['above_mean'],X)# logit model to predict if # of refugees is above the avg.
    
    
    
    
    X_train,X_test,y_train,y_test = train_test_split(X, df['diff'],random_state=5)
    
    AICDiff, varDiff = minAIC(X_train, y_train) #model to estimate gender difference
    #you could add lasso here if you want 
    
    return clf, AICmodel,variables,logitmodel,AICDiff,varDiff

In [39]:
afg_model = myModels('AFG')

D:\anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [30]:
afg_model

(Lasso(alpha=0.05, copy_X=True, fit_intercept=True, max_iter=1000,
    normalize=False, positive=False, precompute=False, random_state=None,
    selection='cyclic', tol=0.0001, warm_start=False),
 Index(['Year', 'ppp', 'gdp', 'life', 'secondary'], dtype='object'),
 (<statsmodels.regression.linear_model.RegressionResultsWrapper at 0x1ed38bb95c8>,
  Index(['Year', 'ppp', 'gdp', 'life', 'secondary'], dtype='object')))

In [31]:
#LASSO COEF
afg_model[0].coef_

array([ 1.67447112e+04,  2.96925432e+02, -9.81692290e-06, -1.59748323e+04,
       -1.56040646e+03])

In [32]:
#AIC, BEST OLS model summary
afg_model[1].summary()

D:\anaconda\lib\site-packages\scipy\stats\stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  total   R-squared (uncentered):                   0.995
Model:                            OLS   Adj. R-squared (uncentered):              0.992
Method:                 Least Squares   F-statistic:                              360.1
Date:                Sat, 22 May 2021   Prob (F-statistic):                    4.44e-10
Time:                        19:05:50   Log-Likelihood:                         -144.69
No. Observations:                  14   AIC:                                      299.4
Df Residuals:                       9   BIC:                                      302.6
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Year        -482.5545    154.623     -3.121      0.012    -832.336    -132.773
ppp          238.3447    152.980      1.558      0.154    -107.720     584.410
gdp        -9.331e-06   5.64e-06     -1.654      0.132   -2.21e-05    3.43e-06
life        2.077e+04   5878.000      3.533      0.006    7471.565    3.41e+04
secondary  -1710.2298    344.493     -4.964      0.001   -2489.526    -930.933
==============================================================================
Omnibus:                        0.067   Durbin-Watson:                   1.586
Prob(Omnibus):                  0.967   Jarque-Bera (JB):                0.225
Skew:                           0.130   Prob(JB):                        0.894
Kurtosis:                       2.437   Cond. No.                     3.85e+10
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.85e+10. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [33]:
#variables that AIC chooses
afg_model[2]

Index(['Year', 'ppp', 'gdp', 'life', 'secondary'], dtype='object')

In [40]:
#AIC, BEST OLS model of gender diff summary
afg_model[4].summary()

D:\anaconda\lib\site-packages\scipy\stats\stats.py:1535: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                   diff   R-squared (uncentered):                   0.982
Model:                            OLS   Adj. R-squared (uncentered):              0.972
Method:                 Least Squares   F-statistic:                              97.02
Date:                Sat, 22 May 2021   Prob (F-statistic):                    1.50e-07
Time:                        19:09:28   Log-Likelihood:                         -109.97
No. Observations:                  14   AIC:                                      229.9
Df Residuals:                       9   BIC:                                      233.1
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Year         -62.2142     12.225     -5.089      0.001     -89.870     -34.558
ppp           58.9072     12.317      4.783      0.001      31.045      86.769
gdp        -2.349e-06   4.51e-07     -5.204      0.001   -3.37e-06   -1.33e-06
life        2536.9863    467.329      5.429      0.000    1479.814    3594.159
secondary   -179.6622     27.748     -6.475      0.000    -242.433    -116.892
==============================================================================
Omnibus:                        9.830   Durbin-Watson:                   2.123
Prob(Omnibus):                  0.007   Jarque-Bera (JB):                5.822
Skew:                           1.205   Prob(JB):                       0.0544
Kurtosis:                       5.042   Cond. No.                     3.55e+10
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.55e+10. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
#LOGIT part is not complete

## Famale Male

In [ ]:
df_ppp = PPP.drop(columns=PPP.columns[[0,1]]).T
df_ppp=sum(np.array(df_ppp).tolist(), [])

df_life = life.drop(columns=life.columns[[0,1]]).T
df_life=sum(np.array(df_life).tolist(), [])

df_gdp = GDP.drop(columns=GDP.columns[[0,1]]).T
df_gdp=sum(np.array(df_gdp).tolist(), [])
df = df2

df['ppp'] = df_ppp
df['gdp'] = df_gdp
df['life'] = df_life
df['secondary'] = df_second

In [22]:
#t test of number of female/male refugees
female = df['female']
male = df['male']

ttest_ind(female, male)

Ttest_indResult(statistic=-1.1186849245951211, pvalue=0.2706838602090723)